# Extracting Date variants in a text file

In this notebook, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this notebook is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, let's find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

In [52]:
import pandas as pd
import re
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

df.head()

0    03/25/93 Total time of visit (in minutes):\n                    
1    6/18/85 Primary Care Doctor:\n                                  
2    sshe plans to move as of 7/8/71 In-Home Services: None\n        
3    7 on 9/27/75 Audit C Score Current:\n                           
4    2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n
dtype: object

In [53]:
df1 = df[0:228]
df2 = df[228:343]
df3 = df[343:455]
df4 = df[455:]

In [54]:
pat1 = r"""(\d{1,2}\b|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|
September|October|November|December)[\/\-\s\.]*(\b\d{1,2}\b|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|
March|April|May|June|July|August|September|October|November|December)[\/\-\s\,\.]*(\b\d{2,4}\b)"""

In [55]:
pat2 = r"""(\d{1,2}\b)[\/\-\s\.]+(Jan[uary]*|Feb[ruary]*|Mar[ch]*|Apr[il]*|May|Jun[e]?|Jul[y]?|Aug[ust]*|Sep[tember]*|
Oct[ober]*|Nov[ember]*|Dec[ember]*)[\/\-\s\,\.]+(\b\d{2,4}\b)"""

In [56]:
pat3 = r"""(Jan[uary]*|Feb[ruary]*|Mar[ch]*|Apr[il]*|May|Jun[e]?|Jul[y]?|Aug[ust]*|Sep[tember]*|Oct[ober]*|Nov[ember]*|
Dec[ember]*)[\/\-\s\,\.]+(\d{2})?[\/\-\s\,\.]+(\d{2,4})"""

In [57]:
def part1(df1):
    final_df = pd.DataFrame()
    pat1 = r"""(\d{1,2}\b|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|
    September|October|November|December)[\/\-\s\.]*(\b\d{1,2}\b|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|
    February|March|April|May|June|July|August|September|October|November|December)[\/\-\s\,\.]*(\b\d{2,4}\b)"""
    pat2 = r"""(\d{1,2}\b)[\/\-\s\.]+(Jan[uary]*|Feb[ruary]*|Mar[ch]*|Apr[il]*|May|Jun[e]?|Jul[y]?|Aug[ust]*|Sep[tember]*|
    Oct[ober]*|Nov[ember]*|Dec[ember]*)[\/\-\s\,\.]+(\b\d{2,4}\b)"""
    pat3 = r"""(Jan[uary]*|Feb[ruary]*|Mar[ch]*|Apr[il]*|May|Jun[e]?|Jul[y]?|Aug[ust]*|Sep[tember]*|Oct[ober]*|Nov[ember]*|
    Dec[ember]*)[\/\-\s\,\.]+(\d{2})?[\/\-\s\,\.]+(\d{2,4})"""
    month_map = {"Jan": "01","Feb": "02","Mar": "03","Apr": "04","May": "05","Jun": "06","Jul": "07","Aug": "08","Sep": "09",
                 "Oct": "10","Nov": "11","Dec": "12"}
    for idx, line in df1.iteritems():
        match = re.search(pat1, line.strip())
        if match is not None:
            date, month, day, year = match[0], match[1], match[2], match[3]
            if day.istitle():
                month, day = day, month
            if len(year) == 2:
                year = "19" + year
            if len(month) == 1:
                month = "0"+month
            if len(day) == 1:
                day = "0" + day
            if month.istitle():
                m = month[:3]
                if m in month_map:
                    month = month_map[m]
                    

            lst = [idx, date, month, day, year]
            temp = pd.DataFrame([lst], columns =["Index", "Date", "Month", "Day", "Year"])
            final_df = final_df.append(temp, ignore_index=True)

        else:
            match = re.search(pat2, line.strip())

        if match is not None:
            date, month, day, year = match[0], match[1], match[2], match[3]
            if day.istitle():
                month, day = day, month
            if len(year) == 2:
                year = "19" + year
            if len(month) == 1:
                month = "0"+month
            if len(day) == 1:
                day = "0" + day 
            if month.istitle():
                m = month[:3]
                if m in month_map:
                    month = month_map[m]
                
            lst = [idx, date, month, day, year]
            temp = pd.DataFrame([lst], columns =["Index", "Date", "Month", "Day", "Year"])
            final_df = final_df.append(temp, ignore_index=True)

        else:
            match = re.search(pat3, line.strip())

        if match is not None:
            date, month, day, year = match[0], match[1], match[2], match[3]
            if day is None:
                day = "01"

            if day.istitle():
                month, day = day, month
            if len(year) == 2:
                year = "19" + year
            if len(month) == 1:
                month = "0"+month
            if len(day) == 1:
                day = "0" + day 
            if month.istitle():
                m = month[:3]
                if m in month_map:
                    month = month_map[m]
                
            lst = [idx, date, month, day, year]
            temp = pd.DataFrame([lst], columns =["Index", "Date", "Month", "Day", "Year"])
            final_df = final_df.append(temp, ignore_index=True)   


        else:
            #print(line.strip())
            pass
    final_df = final_df.drop_duplicates().reset_index().drop(["index"], axis=1)
    return final_df

In [58]:
pat4 = r"""(Jan[uary]*|Feb[ruary]*|Mar[ch]*|Apr[il]*|May|Jun[e]?|Jul[y]?|Aug[ust]*|Sep[tember]*|Oct[ober]*|Nov[ember]*|
Dec[ember]*)[\/\-\s\,\.]+(\d{2,4})"""

In [59]:
def part2(df2):
    month_map = {"Jan": "01","Feb": "02","Mar": "03","Apr": "04","May": "05","Jun": "06","Jul": "07","Aug": "08","Sep": "09",
                 "Oct": "10","Nov": "11","Dec": "12"}
    final_df = part1(df1)
    for idx, line in df2.iteritems():
        pat4 = r"""(Jan[uary]*|Feb[ruary]*|Mar[ch]*|Apr[il]*|May|Jun[e]?|Jul[y]?|Aug[ust]*|Sep[tember]*|Oct[ober]*|Nov[ember]*|
        Dec[ember]*)[\/\-\s\,\.]+(\d{2,4})"""
        match = re.findall(pat4, line)
        if len(match) > 0:
            date, month, year = match[0], match[0][0], match[0][1]
            day = "01"

            if len(year) == 2:
                year = "19" + year

            if month.istitle():
                m = month[:3]
                if m in month_map:
                    month = month_map[m]
                    
            lst = [idx, date, month, day, year]
            temp = pd.DataFrame([lst], columns =["Index", "Date", "Month", "Day", "Year"])
            final_df = final_df.append(temp, ignore_index=True)

        elif len(match) == 0:
            line = "\n\n"+line
            match = re.findall(pat4, line)
            if len(match) > 0:
                date, month, year = match[0], match[0][0], match[0][1]
                day = "01"

                if len(year) == 2:
                    year = "19" + year

                if month.istitle():
                    m = month[:3]
                    if m in month_map:
                        month = month_map[m]
                    
                lst = [idx, date, month.strip(), day, year]
                temp = pd.DataFrame([lst], columns =["Index", "Date", "Month", "Day", "Year"])
                final_df = final_df.append(temp, ignore_index=True)

            elif len(match) == 0:
                match = re.findall(r"(Dec[ember]*)[\/\-\s\,\.]+(\d{2,4})",line)
                if len(match) > 0:
                    date, month, year = match[0], match[0][0], match[0][1]
                    day = "01"

                    if len(year) == 2:
                        year = "19" + year

                    if month.istitle():
                        m = month[:3]
                        if m in month_map:
                            month = month_map[m]
                        
                    lst = [idx, date, month, day, year]
                    temp = pd.DataFrame([lst], columns =["Index", "Date", "Month", "Day", "Year"])
                    final_df = final_df.append(temp, ignore_index=True)

                else:
                    print(line.strip())
    final_df = final_df.drop_duplicates().reset_index().drop(["index"], axis=1)
    return final_df

In [60]:
pat5 = r"(\d{1,2})[\/](\d{4})"

In [61]:
def part3(df3):
    final_df = part2(df2)
    for idx, line in df3.iteritems():
        pat5 = r"(\d{1,2})[\/](\d{4})"
        match = re.findall(pat5, line)

        if len(match) > 0:

            date, month, year = match[0], match[0][0], match[0][1]
            day = "01"
            if len(month) == 1:
                month = "0" + month

            lst = [idx, date, month, day, year]
            temp = pd.DataFrame([lst], columns =["Index", "Date", "Month", "Day", "Year"])
            final_df = final_df.append(temp, ignore_index=True)

        else:
            print(idx,line.strip())
            
    return final_df

In [62]:
pat6 = r"(\d{4})"

In [63]:
def part4(df4):
    final_df = part3(df3)
    for idx, line in df4.iteritems():
        pat6 = r"(\d{4})"
        match = re.findall(pat6, line)
        #print(match)
        if len(match) > 0:
            year = match[0]
            day = "01"
            month = "01"
            date = "01/01/"+year     
            lst = [idx, date, month, day, year]
            temp = pd.DataFrame([lst], columns =["Index", "Date", "Month", "Day", "Year"])
            final_df = final_df.append(temp, ignore_index=True)

        else:
            #print(idx,line.strip())
            pass

    return final_df

In [83]:
def date_sorter():
    month_map = {"Jan": "01","Feb": "02","Mar": "03","Apr": "04","May": "05","Jun": "06","Jul": "07","Aug": "08","Sep": "09",
                 "Oct": "10","Nov": "11","Dec": "12", "January": "01","February":"02","March":"03","April":"04","May":"05", 
                 "June":"06","July":"07","August":"08","September":"09", "October":"10","November":"11","December":"12"}
    df1 = df[0:228]
    df2 = df[228:343]
    df3 = df[343:455]
    df4 = df[455:]
    #final_df = part1(df1)
    #final_df = part2(df2)
    #final_df = part3(df3)
    final_df = part4(df4)
    final_df = final_df.sort_values(by=["Year", "Month", "Day"])
    result = final_df.index
    return result# Your answer here

In [80]:
#final_df = pd.DataFrame()
#a = date_sorter(df)
#len(a)

In [66]:
month_map = {"Jan": "01","Feb": "02","Mar": "03","Apr": "04","May": "05","Jun": "06","Jul": "07","Aug": "08","Sep": "09",
                 "Oct": "10","Nov": "11","Dec": "12", "January": "01","February":"02","March":"03","April":"04","May":"05", 
                 "June":"06","July":"07","August":"08","September":"09", "October":"10","November":"11","December":"12"}

In [1]:
months_3l = "Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec"
months = "January|February|March|April|May|June|July|August|September|October|November|December"
pattern = r"("+months_3l+"|\d{1,2}|"+months+")?[\/\-\s\.]+?(\d{1,2}|"+months_3l+"|"+months+")?([a-zA-Z]{2})?[\/\-\,\.\s]+?(\d{1,4})"


In [23]:
type1 = r"\b(\d{1,2})?\b[\/]?\b(\d{1,2})?\b[\/]\b(\d{2,4})\b"
type2 = r""

In [81]:
#pattern = r"(\d{1,2})?[/-](\d{1,2})?[/-](\d{1,4})"
temp =  df.str.extract(pat1, expand=True)

In [82]:
months = "Jan[uary]*|Feb[ruary]*|Mar[ch]*|Apr[il]*|May|Jun[e]?|Jul[y]?|Aug[ust]*|Sep[tember]*|Oct[ober]*|Nov[ember]*|Dec[ember]*"